In [ ]:
### Using TensorFlow like NumPy

In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import plot
import tensorflow as tf

**Tensors and Operations**

In [2]:
# Set TensorFlow to use CPU only
tf.config.set_visible_devices([], 'GPU')

In [3]:
# A tensor representing a matrix with two rows and three columns of floats
t = tf.constant([[1., 2., 3.], [4., 5., 6]])
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
# Indexing like NumPy
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis]  # without tf.newaxis it would be a 1D array

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
# Operations
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [10]:
# Matrix multiplication
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [11]:
# t + 10 is equivalent to tf.add(t, 10)
tf.add(t, 10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [12]:
# Scalar value
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

**Tensors and NumPy**

In [13]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [14]:
# numpy uses float64 by default. Set dtype to change:
tf.constant(a, dtype=tf.float32)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 5.], dtype=float32)>

In [15]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [16]:
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [17]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [18]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

**Type Conversions**

In [19]:
#tf.constant(2.) + tf.constant(40)

In [20]:
#tf.constant(2.) + tf.constant(40., dtype=tf.float64)

In [21]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)  # tf.constant() is float32 by default

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

**Variables**

In [22]:
# For values that need to change
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [23]:
v + 10  # Returns a constant

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [24]:
v.assign(2 * v)
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [25]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [26]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [27]:
v.scatter_nd_update(
    indices=[[0, 0], [1, 2]], updates=[100., 200.]
)
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

In [28]:
# Direct assignment does not work
#v[1] = [7., 8., 9.]

In [29]:
v[1]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([  8.,  10., 200.], dtype=float32)>

In [30]:
tf.constant("olympiakos")

<tf.Tensor: shape=(), dtype=string, numpy=b'olympiakos'>

In [31]:
tf.constant([ord(c) for c in "olympiakos"])

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([111, 108, 121, 109, 112, 105,  97, 107, 111, 115], dtype=int32)>

### Customizing Models and Training Algorithms

**Custom Losses**

In [32]:
# Implement the Huber loss as a custom loss function with tf operations
# (it is actually available in the tf.keras.losses.Huber class)
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2  # why / 2? See Huber class for details
    linear_loss = tf.abs(error) - 0.5  # why - 0.5? See Huber class for details
    return tf.where(is_small_error, squared_loss, linear_loss)

In [33]:
# extra code – loads, splits and scales the California housing dataset, then
#              creates a simple Keras model

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

input_shape = X_train.shape[1:]

tf.keras.utils.set_random_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          input_shape=input_shape),
    tf.keras.layers.Dense(1),
])

In [34]:
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])
model.fit(X_train_scaled, y_train, epochs=2, validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 0s 679us/step - loss: 0.4827 - mae: 0.8318 - val_loss: 0.3478 - val_mae: 0.6525
Epoch 2/2
363/363 [==============================] - 0s 472us/step - loss: 0.2411 - mae: 0.5412 - val_loss: 0.2641 - val_mae: 0.5484


In [35]:
# When you load a model with custom objects, you need to map the object names to the actual objects
tf.keras.models.save_model(model, "my_model_with_a_custom_loss")

model = tf.keras.models.load_model("my_model_with_a_custom_loss",
                                   custom_objects={"huber_fn": huber_fn})

INFO:tensorflow:Assets written to: my_model_with_a_custom_loss/assets


INFO:tensorflow:Assets written to: my_model_with_a_custom_loss/assets


In [36]:
model.loss

<function __main__.huber_fn(y_true, y_pred)>

In [37]:
# Custom loss function with configurable threshold
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")

In [38]:
tf.keras.models.save_model(model, "my_model_with_a_custom_loss_threshold_2")

model = tf.keras.models.load_model("my_model_with_a_custom_loss_threshold_2",
                                   custom_objects={"huber_fn": create_huber(2.0)})

INFO:tensorflow:Assets written to: my_model_with_a_custom_loss_threshold_2/assets


INFO:tensorflow:Assets written to: my_model_with_a_custom_loss_threshold_2/assets


In [39]:
model.loss

<function __main__.create_huber.<locals>.huber_fn(y_true, y_pred)>

In [40]:
# Better solution: subclass tf.keras.losses.Loss class
# and add 'threshold' to the config dictionary
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config =  super().get_config()
        return {**base_config, "threshold": self.threshold}

In [41]:
my_loss = HuberLoss(2.)
my_loss.get_config()

{'reduction': 'auto', 'name': None, 'threshold': 2.0}

In [42]:
model.compile(loss=my_loss, optimizer="nadam")

In [43]:
tf.keras.models.save_model(model, "my_model_with_a_custom_loss_class")
model = tf.keras.models.load_model("my_model_with_a_custom_loss_class",
                           custom_objects={"HuberLoss": HuberLoss})

INFO:tensorflow:Assets written to: my_model_with_a_custom_loss_class/assets


INFO:tensorflow:Assets written to: my_model_with_a_custom_loss_class/assets


In [44]:
model.loss

In [45]:
my_loss

In [46]:
my_new_loss = HuberLoss.from_config(my_loss.get_config())

In [47]:
my_new_loss.get_config()

{'reduction': 'auto', 'name': None, 'threshold': 2.0}

**Custom Activation Functions, Initializers, Regularizers and Constraints**

In [48]:
# Custom activation function equivalent to tf.keras.activations.softplus()
def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

# Custom Glorot initializer equivalent to tf.keras.initializers.glorot_normal()
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# Custom l1 regularizer equivalent to tf.keras.regularizers.l1(0.01)
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# Custom constraint equivalent to tf.keras.constraints.nonneg()
def my_positive_weights(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [49]:
layer = tf.keras.layers.Dense(1, activation=my_softplus,
                              kernel_initializer=my_glorot_initializer,
                              kernel_regularizer=my_l1_regularizer,
                              kernel_constraint=my_positive_weights)

In [50]:
# Subclass the appropriate class if your custom functions has hyperparameters that
# should be saved along with the model
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {"factor": self.factor}

**Custom Metrics**

In [51]:
# Precision is a stateful metric: it is gradually updated, batch after batch
precision = tf.keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [52]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [53]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [54]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [55]:
precision.reset_states()
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [56]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config
        return {**base_config, "threshold": self.threshold}

**Custom Layers**

In [57]:
# Custom layer without any weights
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))

In [58]:
# Custom layer with weights
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape): 
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": tf.keras.activations.serialize(self.activation)}

In [59]:
# Example of a layer with multiple inputs and multiple outputs
class MyMultiLayer(tf.keras.layers.Layer):
    # ...
    def call(self, X):  # X must be a tuple
        X1, X2 = X
        return X1 + X2, X1 * X2, X1 / X2  # return value is also a tuple

In [60]:
# use the training argument in the call method if your layer differentiates
# between training and inference
class MyGaussianNoise(tf.keras.layers.Layer):  # see also tf.keras.layers.GaussianNoise
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=False):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

**Custom models**

In [61]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="elu",
                                             kernel_initializer="he_normal")
                        for _ in range(n_layers)]
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [62]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation="elu",
                                             kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

**Losses and Metrics based on Model Internals**

In [63]:
# Custom model with a custom reconstruction loss and a corresponding metric
class ReconstructingRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(30, activation="elu",
                                             kernel_initializer="he_normal")
                        for _ in range(5)]
        self.out = tf.keras.layers.Dense(output_dim)
        self.reconstrunction_mean = tf.keras.metrics.Mean(
            name="reconstrunction_error"
        )

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = tf.keras.layers.Dense(n_inputs)

    def call(self, inputs, training=False):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstrunction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstrunction - inputs))
        self.add_loss(0.05 * recon_loss)
        if training:
            # the next two lines could be replaced by self.add_metric(recon_loss)
            result = self.reconstrunction_mean(recon_loss)
            self.add_metric(result)
        return self.out(Z)

In [64]:
# extra code
tf.keras.utils.set_random_seed(42)
model = ReconstructingRegressor(1)
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=5)
y_pred = model.predict(X_test_scaled)

Epoch 1/5


363/363 [==============================] - 1s 603us/step - loss: 0.8010 - reconstrunction_error: 1.2916
Epoch 2/5
363/363 [==============================] - 0s 530us/step - loss: 0.4282 - reconstrunction_error: 0.4761
Epoch 3/5
363/363 [==============================] - 0s 538us/step - loss: 0.4201 - reconstrunction_error: 0.3500
Epoch 4/5
363/363 [==============================] - 0s 580us/step - loss: 0.3660 - reconstrunction_error: 0.2694
Epoch 5/5
162/162 [==============================] - 0s 342us/step


**Computing Gradients Using Autodiff**

In [65]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

# partial derivative wrt w1 = 6 * w1 + 2 * w2
# partial derivative wrt w2 = 2 * w1
# (w1, w2) = (5, 3) -> partial derivative wrt w1 = 36, partial derivative wrt w2 = 10

In [66]:
# Find an approximation of each partial derivative by measuring how much
# the function's output changes when you tweak the corresponding parameter
# by a tiny amount (dy / dx)
w1, w2 = 5, 3
eps = 1e-6
print((f(w1 + eps, w2) - f(w1, w2)) / eps)
print((f(w1, w2 + eps) - f(w1, w2)) / eps)

36.000003007075065
10.000000003174137


In [67]:
# reverse-mode autodiff (see Appendix B for details)
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [68]:
# Raises an Exception if called twice
#tape.gradient(z, w1)

In [69]:
# make it persistent if you want to call it more than once for some reason
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2)
del tape
print(dz_dw1, dz_dw2)

tf.Tensor(36.0, shape=(), dtype=float32) tf.Tensor(10.0, shape=(), dtype=float32)


In [70]:
# By default the tape tracks only Variables
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
gradients

[None, None]

In [71]:
# You can force a tape to track constants
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [72]:
# Use tf.stop_gradient() if you want to stop backpropagation through some part
# of your network
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z,  [w1, w2])  # returns 30., None
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

In [73]:
x = tf.Variable(1e-50)
with tf.GradientTape() as tape:
    z = tf.sqrt(x)

tape.gradient(z, [x])

[<tf.Tensor: shape=(), dtype=float32, numpy=inf>]

In [74]:
# Add a tiny value when computing the function to avoid numerical issues
x = tf.Variable(1e-50)
eps = 1e-6
with tf.GradientTape() as tape:
    z = tf.sqrt(x + eps)

tape.gradient(z, [x])

[<tf.Tensor: shape=(), dtype=float32, numpy=499.99997>]

In [75]:
def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

my_softplus(100.0) # Returns inf instead of the correct result (about 100)

<tf.Tensor: shape=(), dtype=float32, numpy=inf>

In [90]:
# Rewrite the softplus function in a numerically stable way
# so the exp term does not explode
def my_softplus(z):
    return tf.math.log(1 + tf.exp(-tf.abs(z))) + tf.maximum(0., z)

my_softplus(100.)

<tf.Tensor: shape=(), dtype=float32, numpy=100.0>

In [93]:
def my_unstable_softplus(z):
    return tf.math.log(1. + tf.exp(z))

z = tf.Variable(100.)

with tf.GradientTape() as tape:
    v = my_unstable_softplus(z)

tape.gradient(v, z)  # returns NaN

<tf.Tensor: shape=(), dtype=float32, numpy=nan>

In [94]:
# A function may have numerically unstable gradients.
# Use the @tf.custom_gradient decorator to tell Tensorflow how to compute the gradient
@tf.custom_gradient
# here we write numerically stable versions both for the function itself
# and for its gradients
def my_softplus(z):
    def my_softplus_gradients(grads):  # grads = backpropagated from upper layers
        return grads * (1 - 1 / (1 + tf.exp(z)))  # stable gradients of softplus
    
    result = tf.math.log(1 + tf.exp(-tf.abs(z))) + tf.maximum(0., z)
    return result, my_softplus_gradients

In [95]:
z = tf.Variable(100.)

with tf.GradientTape() as tape:
    v = my_softplus(z)

tape.gradient(v, z)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

### Custom Training Loops

In [118]:
# Build a simple model
l2_reg = tf.keras.regularizers.l2(0.05)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal",
                          kernel_regularizer=l2_reg),
    tf.keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [98]:
# A simple function that samples a batch of instances from the training set
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [103]:
# Utility function
def print_status_bar(step, total, loss, metrics=None):
    metrics = " - ".join([f"{m.name}: {m.result():.4f}"
                          for m in [loss] + (metrics or [])])
    end = "" if step < total else "\n"
    print(f"\r{step}/{total} - " + metrics, end=end)

In [116]:
# Define some hyperparameters
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error
mean_loss = tf.keras.metrics.Mean(name="mean_loss")
metrics = [tf.keras.metrics.MeanAbsoluteError()]

In [119]:
# Build the custom training loop
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            # model.losses -> one regularization loss per layer
            loss = tf.add_n([main_loss] + model.losses)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)

        print_status_bar(step, n_steps, mean_loss, metrics)

    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/5
362/362 - mean_loss: 2.8848 - mean_absolute_error: 0.6136
Epoch 2/5
362/362 - mean_loss: 1.6322 - mean_absolute_error: 0.5303
Epoch 3/5
362/362 - mean_loss: 1.0789 - mean_absolute_error: 0.4990
Epoch 4/5
362/362 - mean_loss: 0.8132 - mean_absolute_error: 0.4871
Epoch 5/5
362/362 - mean_loss: 0.7092 - mean_absolute_error: 0.4897


### TensorFlow Functions and Graphs

In [120]:
def cube(x):
    return x ** 3

In [121]:
cube(2)

8

In [122]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [123]:
cube(tf.constant([2, 3]))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 8, 27], dtype=int32)>

In [124]:
# Convert this function to a TensorFlow function
tf_cube = tf.function(cube)
tf_cube

In [125]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [126]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [127]:
tf_cube(tf.constant([2, 3]))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 8, 27], dtype=int32)>

In [128]:
# Use tf.function as a decorator
@tf.function
def tf_cube(x):
    return x ** 3

In [129]:
# The original function is still available
tf_cube.python_function(2)

8

**Autograph and Tracing**

In [132]:
print(tf.autograph.to_code(tf_cube.python_function))

def tf__tf_cube(x):
    with ag__.FunctionScope('tf_cube', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        try:
            do_return = True
            retval_ = ag__.ld(x) ** 3
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

